In [56]:
import pathlib
import chess
import chess.pgn
import chess.engine
import numpy as np
import h5py

In [57]:
pgnGames = list(pathlib.Path('').glob('*.pgn'))

In [58]:
# General space saving algorithms that are often used in chess programming sites

def board_to_arr(board: chess.Board):
    black, white = board.occupied_co

    bitboards = np.array([
        black & board.pawns,
        black & board.knights,
        black & board.bishops,
        black & board.rooks,
        black & board.queens,
        black & board.kings,
        white & board.pawns,
        white & board.knights,
        white & board.bishops,
        white & board.rooks,
        white & board.queens,
        white & board.kings,
    ], dtype=np.uint64)

    return bitboards

def bitboards_to_array(bb: np.ndarray) -> np.ndarray:
    bb = np.asarray(bb, dtype=np.uint64)[:, np.newaxis]
    s = 8 * np.arange(7, -1, -1, dtype=np.uint64)
    b = (bb >> s).astype(np.uint8)
    b = np.unpackbits(b, bitorder="little")
    return b.reshape(-1, 8, 8)


In [59]:
engine = chess.engine.SimpleEngine.popen_uci("engines/stockfish-macos-x86-64-bmi2")

def convert_to_winrate(acpl):
    
    return (50 + 50 * (2 / (1 + np.exp(-0.00368208 * acpl)) - 1)) * 0.01

def evaluate_game(game, think_time=0.1, verbose=False):
    
    single_game = []
    evaluations = []
    
    board = game.board()

    for move in game.mainline_moves():
    
        # Evaluate the current position 
        result = engine.analyse(board, chess.engine.Limit(time=think_time))
        
        if result['score'].turn: 
            try:
                objective_eval = result['score'].relative.cp
                evaluations.append(convert_to_winrate(objective_eval))
            except:
                if result['score'].relative.moves > 0:
                    evaluations.append(1)
                else:
                    evaluations.append(0)
        else:
            try:
                objective_eval = result['score'].relative.cp * -1
                evaluations.append(convert_to_winrate(objective_eval))
            except:
                if result['score'].relative.moves < 0:
                    evaluations.append(1)
                else:
                    evaluations.append(0)       
        
        # Push the game state to new
        board.push_uci(move.uci())
        single_game.append(board_to_arr(board))
    
    # Evaluate final position once more to find accuracy of last move
    result = engine.analyse(board, chess.engine.Limit(time=think_time))
    evaluations.append(convert_to_winrate(objective_eval))
    if verbose:
        print(evaluations)
    
    winrate_diff = [abs(x - evaluations[i - 1]) for i, x in enumerate(evaluations)][1:]

    return single_game, winrate_diff
    

### Generate Dataset

In [60]:
X = []
y = []

for g in range(len(pgnGames)):
    pgn = open(pgnGames[g])
    
    for k in range(5000):  # 190,000 assures all games are looked at.
        
        try:
            game = chess.pgn.read_game(pgn)

            board_history, winrate_history = evaluate_game(game, think_time=0.025, verbose=False)
            X += board_history
            y += winrate_history
            print(f"{k+1} games analyzed, dataset size {len(X)}")
        except Exception as e:
            print(f"{e}")
            pass
          

1 games analyzed, dataset size 55
2 games analyzed, dataset size 192
3 games analyzed, dataset size 251
4 games analyzed, dataset size 383
5 games analyzed, dataset size 481
6 games analyzed, dataset size 572
7 games analyzed, dataset size 625
8 games analyzed, dataset size 670
9 games analyzed, dataset size 695
10 games analyzed, dataset size 816
11 games analyzed, dataset size 869
12 games analyzed, dataset size 991
13 games analyzed, dataset size 1059
14 games analyzed, dataset size 1115
15 games analyzed, dataset size 1210
16 games analyzed, dataset size 1302
17 games analyzed, dataset size 1372
18 games analyzed, dataset size 1432
19 games analyzed, dataset size 1507
20 games analyzed, dataset size 1583
21 games analyzed, dataset size 1663
22 games analyzed, dataset size 1707
23 games analyzed, dataset size 1792
24 games analyzed, dataset size 1949
25 games analyzed, dataset size 2001
26 games analyzed, dataset size 2069
27 games analyzed, dataset size 2136
28 games analyzed, data

218 games analyzed, dataset size 18287
219 games analyzed, dataset size 18362
220 games analyzed, dataset size 18448
221 games analyzed, dataset size 18520
222 games analyzed, dataset size 18578
223 games analyzed, dataset size 18683
224 games analyzed, dataset size 18735
225 games analyzed, dataset size 18836
226 games analyzed, dataset size 18933
227 games analyzed, dataset size 18992
228 games analyzed, dataset size 19062
229 games analyzed, dataset size 19115
230 games analyzed, dataset size 19168
231 games analyzed, dataset size 19426
232 games analyzed, dataset size 19536
233 games analyzed, dataset size 19642
234 games analyzed, dataset size 19705
235 games analyzed, dataset size 19785
236 games analyzed, dataset size 19877
237 games analyzed, dataset size 19936
238 games analyzed, dataset size 19970
239 games analyzed, dataset size 20058
240 games analyzed, dataset size 20128
241 games analyzed, dataset size 20184
242 games analyzed, dataset size 20241
243 games analyzed, datas

429 games analyzed, dataset size 36170
430 games analyzed, dataset size 36251
431 games analyzed, dataset size 36336
432 games analyzed, dataset size 36385
433 games analyzed, dataset size 36542
434 games analyzed, dataset size 36586
435 games analyzed, dataset size 36678
436 games analyzed, dataset size 36747
437 games analyzed, dataset size 36844
438 games analyzed, dataset size 36932
439 games analyzed, dataset size 37016
440 games analyzed, dataset size 37099
441 games analyzed, dataset size 37121
442 games analyzed, dataset size 37258
443 games analyzed, dataset size 37318
444 games analyzed, dataset size 37375
445 games analyzed, dataset size 37468
446 games analyzed, dataset size 37580
447 games analyzed, dataset size 37719
448 games analyzed, dataset size 37771
449 games analyzed, dataset size 37876
450 games analyzed, dataset size 37971
451 games analyzed, dataset size 38121
452 games analyzed, dataset size 38235
453 games analyzed, dataset size 38291
454 games analyzed, datas

640 games analyzed, dataset size 53150
641 games analyzed, dataset size 53215
642 games analyzed, dataset size 53299
643 games analyzed, dataset size 53372
644 games analyzed, dataset size 53422
645 games analyzed, dataset size 53513
646 games analyzed, dataset size 53551
647 games analyzed, dataset size 53673
648 games analyzed, dataset size 53721
649 games analyzed, dataset size 53877
650 games analyzed, dataset size 53982
651 games analyzed, dataset size 54047
652 games analyzed, dataset size 54061
653 games analyzed, dataset size 54212
654 games analyzed, dataset size 54282
655 games analyzed, dataset size 54380
656 games analyzed, dataset size 54497
657 games analyzed, dataset size 54577
658 games analyzed, dataset size 54647
659 games analyzed, dataset size 54722
660 games analyzed, dataset size 54841
661 games analyzed, dataset size 54953
662 games analyzed, dataset size 55115
663 games analyzed, dataset size 55248
664 games analyzed, dataset size 55352
665 games analyzed, datas

851 games analyzed, dataset size 71901
852 games analyzed, dataset size 72068
853 games analyzed, dataset size 72112
854 games analyzed, dataset size 72171
855 games analyzed, dataset size 72264
856 games analyzed, dataset size 72404
857 games analyzed, dataset size 72518
858 games analyzed, dataset size 72575
859 games analyzed, dataset size 72600
860 games analyzed, dataset size 72669
861 games analyzed, dataset size 72740
862 games analyzed, dataset size 72864
863 games analyzed, dataset size 72948
864 games analyzed, dataset size 73024
865 games analyzed, dataset size 73109
866 games analyzed, dataset size 73157
867 games analyzed, dataset size 73274
868 games analyzed, dataset size 73401
869 games analyzed, dataset size 73517
870 games analyzed, dataset size 73585
871 games analyzed, dataset size 73617
872 games analyzed, dataset size 73691
873 games analyzed, dataset size 73806
874 games analyzed, dataset size 73891
875 games analyzed, dataset size 73954
876 games analyzed, datas

...

4064 games analyzed, dataset size 335111
4065 games analyzed, dataset size 335154
4066 games analyzed, dataset size 335269
4067 games analyzed, dataset size 335332
4068 games analyzed, dataset size 335424
4069 games analyzed, dataset size 335492
4070 games analyzed, dataset size 335603
4071 games analyzed, dataset size 335690
4072 games analyzed, dataset size 335738
4073 games analyzed, dataset size 335749
4074 games analyzed, dataset size 335825
4075 games analyzed, dataset size 335875
4076 games analyzed, dataset size 335944
4077 games analyzed, dataset size 335979
4078 games analyzed, dataset size 336039
4079 games analyzed, dataset size 336089
4080 games analyzed, dataset size 336191
4081 games analyzed, dataset size 336248
4082 games analyzed, dataset size 336283
4083 games analyzed, dataset size 336350
4084 games analyzed, dataset size 336495
4085 games analyzed, dataset size 336562
4086 games analyzed, dataset size 336644
4087 games analyzed, dataset size 336750
4088 games analy

4264 games analyzed, dataset size 351210
4265 games analyzed, dataset size 351273
4266 games analyzed, dataset size 351390
4267 games analyzed, dataset size 351468
4268 games analyzed, dataset size 351520
4269 games analyzed, dataset size 351634
4270 games analyzed, dataset size 351682
4271 games analyzed, dataset size 351800
4272 games analyzed, dataset size 351841
4273 games analyzed, dataset size 351923
4274 games analyzed, dataset size 352046
4275 games analyzed, dataset size 352169
4276 games analyzed, dataset size 352311
4277 games analyzed, dataset size 352407
4278 games analyzed, dataset size 352500
4279 games analyzed, dataset size 352575
4280 games analyzed, dataset size 352682
4281 games analyzed, dataset size 352772
4282 games analyzed, dataset size 352901
4283 games analyzed, dataset size 352933
4284 games analyzed, dataset size 352984
4285 games analyzed, dataset size 353049
4286 games analyzed, dataset size 353095
4287 games analyzed, dataset size 353158
4288 games analy

4464 games analyzed, dataset size 367000
4465 games analyzed, dataset size 367071
4466 games analyzed, dataset size 367132
4467 games analyzed, dataset size 367215
4468 games analyzed, dataset size 367312
4469 games analyzed, dataset size 367352
4470 games analyzed, dataset size 367440
4471 games analyzed, dataset size 367520
4472 games analyzed, dataset size 367608
4473 games analyzed, dataset size 367691
4474 games analyzed, dataset size 367740
4475 games analyzed, dataset size 367798
4476 games analyzed, dataset size 367852
4477 games analyzed, dataset size 367958
4478 games analyzed, dataset size 368016
4479 games analyzed, dataset size 368067
4480 games analyzed, dataset size 368142
4481 games analyzed, dataset size 368245
4482 games analyzed, dataset size 368288
4483 games analyzed, dataset size 368371
4484 games analyzed, dataset size 368494
4485 games analyzed, dataset size 368581
4486 games analyzed, dataset size 368635
4487 games analyzed, dataset size 368674
4488 games analy

4664 games analyzed, dataset size 383328
4665 games analyzed, dataset size 383395
4666 games analyzed, dataset size 383475
4667 games analyzed, dataset size 383556
4668 games analyzed, dataset size 383630
4669 games analyzed, dataset size 383734
4670 games analyzed, dataset size 383813
4671 games analyzed, dataset size 383908
4672 games analyzed, dataset size 384014
4673 games analyzed, dataset size 384133
4674 games analyzed, dataset size 384201
4675 games analyzed, dataset size 384344
4676 games analyzed, dataset size 384469
4677 games analyzed, dataset size 384524
4678 games analyzed, dataset size 384593
4679 games analyzed, dataset size 384643
4680 games analyzed, dataset size 384667
4681 games analyzed, dataset size 384683
4682 games analyzed, dataset size 384757
4683 games analyzed, dataset size 384809
4684 games analyzed, dataset size 384862
4685 games analyzed, dataset size 384911
4686 games analyzed, dataset size 385010
4687 games analyzed, dataset size 385049
4688 games analy

4864 games analyzed, dataset size 399884
4865 games analyzed, dataset size 399951
4866 games analyzed, dataset size 400016
4867 games analyzed, dataset size 400048
4868 games analyzed, dataset size 400105
4869 games analyzed, dataset size 400185
4870 games analyzed, dataset size 400296
4871 games analyzed, dataset size 400377
4872 games analyzed, dataset size 400457
4873 games analyzed, dataset size 400547
4874 games analyzed, dataset size 400647
4875 games analyzed, dataset size 400705
4876 games analyzed, dataset size 400827
4877 games analyzed, dataset size 400902
4878 games analyzed, dataset size 401004
4879 games analyzed, dataset size 401062
4880 games analyzed, dataset size 401191
4881 games analyzed, dataset size 401300
4882 games analyzed, dataset size 401391
4883 games analyzed, dataset size 401507
4884 games analyzed, dataset size 401636
4885 games analyzed, dataset size 401698
4886 games analyzed, dataset size 401736
4887 games analyzed, dataset size 401858
4888 games analy

In [61]:
X = np.dstack(X)
y = np.array(y, dtype='float32')
X = np.rollaxis(X, -1).reshape(-1, 12)

In [62]:
X.shape, y.shape

((411343, 12), (411343,))

In [63]:
print(bitboards_to_array(X[0]))

[[[0 0 0 0 0 0 0 0]
  [1 1 1 1 1 1 1 1]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[0 1 0 0 0 0 1 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[0 0 1 0 0 1 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[1 0 0 0 0 0 0 1]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[0 0 0 1 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[0 0 0 0 1 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]]

 [[0 0 0 0 0 0 0 0]
  [0 0 0

In [64]:
with h5py.File('positions.h5', 'w') as hf:
    hf.create_dataset("X", data=X, compression='gzip', compression_opts=5)
    hf.create_dataset("y", data=y, compression='gzip', compression_opts=5)

In [65]:
print(max(y))

1.0


In [66]:
sorted(y)[::-1][:50]

[1.0,
 1.0,
 1.0,
 1.0,
 0.9660068,
 0.9660068,
 0.96271896,
 0.9456945,
 0.93609005,
 0.92987823,
 0.92258793,
 0.91345197,
 0.91345197,
 0.91205704,
 0.8967275,
 0.8967275,
 0.89361805,
 0.8867819,
 0.87760174,
 0.87561035,
 0.87399733,
 0.86821014,
 0.86407167,
 0.86219907,
 0.8596773,
 0.85955316,
 0.8595424,
 0.85910803,
 0.8586618,
 0.8586618,
 0.85549045,
 0.8536745,
 0.85283273,
 0.85193205,
 0.8513598,
 0.8513598,
 0.8472463,
 0.84228736,
 0.84228736,
 0.8398263,
 0.8398263,
 0.83898693,
 0.83531815,
 0.83531815,
 0.8322565,
 0.8322565,
 0.8322565,
 0.8296706,
 0.82731193,
 0.82652575]